In [ ]:
!pip install textattack
!pip install tensorflow-text
!pip install datasets
# !pip install --upgrade datasets

In [2]:
from google.colab import drive
import os
drive.mount('/content/drive')
### Change directory to wherever sst and cola csv files are located ###
os.chdir('/content/drive/MyDrive/263 Adversarial')

Mounted at /content/drive


In [25]:
import pandas as pd

sst_tr_dataset = pd.read_csv('sst_train_data.csv')
sst_val_dataset = pd.read_csv('sst_val_data.csv')
sst_test_dataset = pd.read_csv('sst_test_data.csv')

cola_tr_dataset = pd.read_csv('cola_train_data.csv')
cola_val_dataset = pd.read_csv('cola_val_data.csv')
cola_test_dataset = pd.read_csv('cola_test_data.csv')

In [4]:
sst_val_dataset.sample(5)

,sentences,labels
464,"essentially an exceptionally well-written , we...",1
539,"This is wild surreal stuff , but brilliant and...",1
887,The so-inept - it 's - surreal dubbing ( featu...,0
1066,A beautifully made piece of unwatchable drivel .,0
936,What is 100 % missing here is a script of even...,0


In [5]:
cola_val_dataset.sample(5)

,sentences,labels
48,She has kissed her.,1
257,Anson demonized old,0
104,Medea tried her to leave.,0
683,The intrepid pirate and the fearful captain's ...,1
838,Show me letters!,1


In [6]:
from transformers import AutoModelForSequenceClassification, TFAutoModelForSequenceClassification, AutoTokenizer, pipeline
# import textattack

bert_sst = AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-SST-2")
bert_sst_tokenizer = AutoTokenizer.from_pretrained("textattack/bert-base-uncased-SST-2")

bert_cola = AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-CoLA")
bert_cola_tokenizer = AutoTokenizer.from_pretrained('textattack/bert-base-uncased-CoLA')

roberta_sst = AutoModelForSequenceClassification.from_pretrained("textattack/roberta-base-SST-2")
roberta_sst_tokenizer = AutoTokenizer.from_pretrained("textattack/roberta-base-SST-2")

roberta_cola = AutoModelForSequenceClassification.from_pretrained("textattack/roberta-base-CoLA")
roberta_cola_tokenizer = AutoTokenizer.from_pretrained('textattack/roberta-base-CoLA')

# from textattack.models.helpers.lstm_for_classification import LSTMForClassification
# lstm = LSTMForClassification()
# lstm_sst = lstm.from_pretrained('lstm-sst2')
# lstm_cola = lstm.from_pretrained('lstm-cola')

Downloading:   0%|          | 0.00/477 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

Some weights of the model checkpoint at textattack/roberta-base-CoLA were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [ ]:
import textattack
import transformers
from textattack import Attack, AttackArgs, Attacker
from textattack.datasets import Dataset

# Construct our four components for `Attack`
from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.constraints.semantics import WordEmbeddingDistance

from textattack.transformations import WordSwapEmbedding
from textattack.search_methods import GreedyWordSwapWIR

new_dataset = []
for sentence, label in zip(sst_val_dataset['sentences'], sst_val_dataset['labels']):
  new_dataset.append((sentence, int(label)))

sst_dataset = Dataset(new_dataset)

new_dataset = []
for sentence, label in zip(cola_val_dataset['sentences'], cola_val_dataset['labels']):
  new_dataset.append((sentence, int(label)))

cola_dataset = Dataset(new_dataset)

In [ ]:
model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(bert_sst, bert_sst_tokenizer)

goal_function = textattack.goal_functions.UntargetedClassification(model_wrapper)
constraints = [RepeatModification(),
  StopwordModification(),
  WordEmbeddingDistance(min_cos_sim=0.8)]

transformation = WordSwapEmbedding(max_candidates=50)
search_method = GreedyWordSwapWIR(wir_method="delete")

# Construct the actual attack
attack = Attack(goal_function, constraints, transformation, search_method)

attack_args = AttackArgs(
    num_examples=1000,
)

attacker = Attacker(attack, sst_dataset, attack_args)
attack_set = attacker.attack_dataset()

successful_attacks = []
labels = []
for result in attack_set:
  if type(result) == textattack.attack_results.successful_attack_result.SuccessfulAttackResult:
    label = int(result.goal_function_result_str()[0])
    successful_attacks.append(result.perturbed_text())
    labels.append(label)

adversarial_df = pd.DataFrame({'sentences':successful_attacks, 'labels':labels})
adversarial_df.to_csv("bert_adversarial_sst.csv", index=False)
adversarial_df.head()

In [ ]:
model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(bert_cola, bert_cola_tokenizer)

goal_function = textattack.goal_functions.UntargetedClassification(model_wrapper)
constraints = [RepeatModification(),
  StopwordModification(),
  WordEmbeddingDistance(min_cos_sim=0.8)]

transformation = WordSwapEmbedding(max_candidates=50)
search_method = GreedyWordSwapWIR(wir_method="delete")

# Construct the actual attack
attack = Attack(goal_function, constraints, transformation, search_method)

attack_args = AttackArgs(
    num_examples=1000,
)

attacker = Attacker(attack, cola_dataset, attack_args)
attack_set = attacker.attack_dataset()

successful_attacks = []
labels = []
for result in attack_set:
  if type(result) == textattack.attack_results.successful_attack_result.SuccessfulAttackResult:
    label = int(result.goal_function_result_str()[0])
    successful_attacks.append(result.perturbed_text())
    labels.append(label)

adversarial_df = pd.DataFrame({'sentences':successful_attacks, 'labels':labels})
adversarial_df.to_csv("bert_adversarial_cola.csv", index=False)
adversarial_df.head()

In [ ]:
model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(roberta_sst, roberta_sst_tokenizer)

goal_function = textattack.goal_functions.UntargetedClassification(model_wrapper)
constraints = [RepeatModification(),
  StopwordModification(),
  WordEmbeddingDistance(min_cos_sim=0.8)]

transformation = WordSwapEmbedding(max_candidates=50)
search_method = GreedyWordSwapWIR(wir_method="delete")

# Construct the actual attack
attack = Attack(goal_function, constraints, transformation, search_method)

attack_args = AttackArgs(
    num_examples=1000,
)

attacker = Attacker(attack, sst_dataset, attack_args)
attack_set = attacker.attack_dataset()

successful_attacks = []
labels = []
for result in attack_set:
  if type(result) == textattack.attack_results.successful_attack_result.SuccessfulAttackResult:
    label = int(result.goal_function_result_str()[0])
    successful_attacks.append(result.perturbed_text())
    labels.append(label)

adversarial_df = pd.DataFrame({'sentences':successful_attacks, 'labels':labels})
adversarial_df.to_csv("roberta_adversarial_sst.csv", index=False)
adversarial_df.head()

In [ ]:
model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(roberta_cola, roberta_cola_tokenizer)

goal_function = textattack.goal_functions.UntargetedClassification(model_wrapper)
constraints = [RepeatModification(),
  StopwordModification(),
  WordEmbeddingDistance(min_cos_sim=0.8)]

transformation = WordSwapEmbedding(max_candidates=50)
search_method = GreedyWordSwapWIR(wir_method="delete")

# Construct the actual attack
attack = Attack(goal_function, constraints, transformation, search_method)

attack_args = AttackArgs(
    num_examples=1000,
)

attacker = Attacker(attack, cola_dataset, attack_args)
attack_set = attacker.attack_dataset()

successful_attacks = []
labels = []
for result in attack_set:
  if type(result) == textattack.attack_results.successful_attack_result.SuccessfulAttackResult:
    label = int(result.goal_function_result_str()[0])
    successful_attacks.append(result.perturbed_text())
    labels.append(label)

adversarial_df = pd.DataFrame({'sentences':successful_attacks, 'labels':labels})
adversarial_df.to_csv("roberta_adversarial_cola.csv", index=False)
adversarial_df.head()

Data Augmentation

In [ ]:
from textattack.augmentation import WordNetAugmenter, EmbeddingAugmenter, DeletionAugmenter, CharSwapAugmenter, CheckListAugmenter, CLAREAugmenter, BackTranslationAugmenter
import random
from functools import partial
from datasets import concatenate_datasets
from tqdm import tqdm

augmenters = [WordNetAugmenter(), EmbeddingAugmenter(), DeletionAugmenter(), CharSwapAugmenter(), BackTranslationAugmenter()]
# augmenters = [WordNetAugmenter()]
# augmenter_names = ["wordnet", "embedding", "deletion", "charswap", "backtranslation"]
augmented_dataset_size = 1500
augmented_dataset = sst_tr_dataset.iloc[:augmented_dataset_size].copy()
chunk_size = augmented_dataset_size // len(augmenters)
# datasets = [sst_tr_dataset_eval.iloc[i*chunk_size : (i+1)*chunk_size] for i in range(len(augmenters))]

# for i in tqdm(range(len(augmenters))):
augmenter_index = 0
for i in tqdm(range(augmented_dataset_size)):
  if i % chunk_size == 0:
    augmenter = augmenters[augmenter_index]
    augmenter_index += 1

  original_sentence = augmented_dataset.at[i, 'sentences']
  try:
    augmented_sentence = augmenter.augment(original_sentence)
    augmented_dataset.at[i, 'sentences'] = augmented_sentence
  except:
    print(original_sentence)
    continue
# augmented_dataset = pd.concat(datasets)
augmented_dataset.to_csv("augmented_sst.csv", index=False)
print(len(augmented_dataset))
  

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


 80%|███████▉  | 1197/1500 [02:43<00:09, 33.19it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3407: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
 82%|████████▏ | 1236/1500 [13:28<50:08, 11.40s/it]  

In [ ]:
# from textattack.augmentation import WordNetAugmenter, EmbeddingAugmenter, DeletionAugmenter, CharSwapAugmenter, CheckListAugmenter, CLAREAugmenter, BackTranslationAugmenter
# import random
# from functools import partial
# from datasets import concatenate_datasets
# from tqdm import tqdm

augmenters = [WordNetAugmenter(), EmbeddingAugmenter(), DeletionAugmenter(), CharSwapAugmenter(), BackTranslationAugmenter()]
# augmenters = [WordNetAugmenter()]
# augmenter_names = ["wordnet", "embedding", "deletion", "charswap", "backtranslation"]
augmented_dataset_size = 1500
augmented_dataset = cola_tr_dataset.iloc[:augmented_dataset_size].copy()
chunk_size = augmented_dataset_size // len(augmenters)
# datasets = [sst_tr_dataset_eval.iloc[i*chunk_size : (i+1)*chunk_size] for i in range(len(augmenters))]

# for i in tqdm(range(len(augmenters))):
augmenter_index = 0
for i in tqdm(range(augmented_dataset_size)):
  if i % chunk_size == 0:
    augmenter = augmenters[augmenter_index]
    augmenter_index += 1

  original_sentence = augmented_dataset.at[i, 'sentences']
  try:
    augmented_sentence = augmenter.augment(original_sentence)
    augmented_dataset.at[i, 'sentences'] = augmented_sentence
  except:
    print(original_sentence)
    continue
# augmented_dataset = pd.concat(datasets)
augmented_dataset.to_csv("augmented_cola.csv", index=False)
print(len(augmented_dataset))
  